# The Creator<br>

 #### This is a deep reinforcement learning playground in which one or multiple neural netwoks in a generative arrangement seed the initial frame with **n** live cells (seeds). The artificially seeded frames are then evolved for **t** generations. The loss of becomes a function of the number of live cells at generation t. Non-adherence to the restriction of the number of seeds (n) needs to be punished severly in order for the game to work.
  

In [36]:
### CONWAY'S GAME OF LIFE 
### Welcome to Conway's Game of Life 
### the 3 rules of the game are:
#1. Any live cell with two or three live neighbours survives.
#2. Any dead cell with three live neighbours becomes a live cell.
#3. All other live cells die in the next generation. Similarly, all other dead cells stay dead.

import numpy as np
from matplotlib import colors
import matplotlib.pyplot as plt
import pylab as pl
import random
import pandas as pd
import torch
import torchvision
from tqdm.notebook import tqdm
from functions.data import Data, torch_evolve, spawner
import wandb
device = torch.device("cuda")

frame_shape = (10,10)


In [ ]:
class Generator(torch.nn.Module):
    def __init__(self, n_frames : int, p : float):
        super(Generator, self).__init__()
        layers = 4
        assert n_frames % layers == 0, f"n_frames must be divisible by the number of layers ({layers})"
        self.relu = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p)
        self.tconv1 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(1, n_frames//layers*1, kernel_size=5, stride=1, padding=1),
            torch.nn.BatchNorm2d(n_frames//layers*1),
            self.relu,
            self.dropout)
        self.tconv2 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(n_frames//layers*1, n_frames//layers*2, kernel_size=5, stride=1, padding=1),
            torch.nn.BatchNorm2d(n_frames//layers*2),
            self.relu,
            self.dropout)
        self.tconv3 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(n_frames//layers*2, n_frames//layers*3, kernel_size=5, stride=1, padding=1),
            torch.nn.BatchNorm2d(n_frames//layers*3),
            self.relu,
            self.dropout)
        self.tconv4 = torch.nn.Sequential(
            torch.nn.ConvTranspose2d(n_frames//layers*3, n_frames//layers*4, kernel_size=6, stride=1, padding=1),
            torch.nn.BatchNorm2d(n_frames//layers*4),
            self.relu,)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.tconv1(x)
        x = self.tconv2(x)
        x = self.tconv3(x)
        x = self.tconv4(x)
        x = self.sigmoid(x)
        return x



In [24]:
def reward(input, output, output_evolved, order=2):
    return (((input-torch.abs(torch.sum(output, dim(-1, -2))-input))/input)**order) * (torch.sum(evolved_output, dim(-1, -2))-input)/input

10


In [ ]:
def evolve(frame, generations):
    for i in range(generations):
        frame = torch_evolve(frame)
    return frame

In [ ]:
generator = Generator(256, 0.1).to(device)
optim = torch.optim.Adam(generator.parameters(), lr=0.001)
epochs = 100
batch_size = 32

for epoch in tqdm(range(epochs)):
    for batch in tqdm(range(batch_size)):
        input = torch.randint(5, 20, batch_size)
        torch repaet